### INST326 OOP Project 03

Rename this notebook, replacing "_Assignment" with "_YourName"<br>
Insert Signature Block Here

#### Group 54: Christian Sorensen, Tamunosaki Danagogo, Marcos Alvarado
> INST326
> Project 3
> 11/1/2024
#### Honor Pledge
> I pledge that the work contained in this assignment is my own, and that I have complied with University and course policies on academic integrity and AI use.


### The Project
Everyone will do the same project this time. This is a group project, so you must work in your assigned groups. Include the link to your group's GitHub repository (one link per group). Use comments in your code to document your solution. If you need to write comments to the grader, add a markdown cell immediately above your code solution and add your comments there. Be sure to read and follow all the requirements and the Notebook Instructions at the bottom of this notebook. Your grade may depend on it!

#### 1. A Scheduling Program
>  My wife is responsible for scheduling caregivers for her 93 year-old mother. Currently she writes out the schedule on a monthly calendar and photocopies it for everyone. I want all of you to help me write a program to help her with scheduling. While this is a specific application, this program will be broadly useful and adaptable to any scheduling needs for small businesses, clubs, and more.

#### Requirements
>  Care is required 12 hours per day, 7 days a week. There are two shifts each day: 7:00 AM - 1:00 PM, and 1:00 PM to 7:00 PM. There are a total of 8 caregivers. Some are family members and some are paid. Each caregiver has their own availability for shifts that is generally the same from month to month, but there are exceptions for work, vacations, and other responsibilities. Your program should do the following:
> 1. Manage caregivers and their schedules. Attributes include: name, phone, email, pay rate, and hours.
> 2. Each caregiver should have their own availability schedule where they can indicate their availability for each shift. Availability categories are 'preferred', 'available' (default), and 'unavailable'.
> 3. Create a care schedule that covers AM and PM shifts and displays caregiver names on a calendar (see example). The schedule should accomodate caregivers' individual schedules and availability preferences. The python calendar module provides options for creating HTML calendars. Sample code for the HTML calendar is in the project folder.
> 4. Paid caregivers are paid weekly at $20/hr. Your program should calculate weekly pay based on assigned hours. Provide a separate pay report that lists weekly (gross: hours x rate) amounts to each caregiver, along with weekly and monthly totals. The report can be a text document, or presented in GUI or HTML format. 

#### Group Requirements
>  1. Your submitted project should follow OOP principles like abstraction, encapsulation, inheritance, and polymorphism as appropriate. Your program should use classes. 
>  2. Select a group leader who will host the group's project repository on their GitHub.
>  3. Create the group repository and add a main program document. See example.
>  4. Create branches off the main program for each group member, and assign part of the program to each member.
>  5. Each member should work on their branch.
>  6. When each member is finished, merge the branches back into the main program. You may use 'merge' or 'pull requests', your choice.
>  7. iterate and debug as necessary.

#### Working with HTML
> Since this is a course on python, not HTML, you are not expected to know HTML. Therefore, you may copy applicable portions of the sample code or use AI to write the HTML portions of your application. Ypu should write the main python code yourself.


#### What you need to turn in
>  This is a group project. There will be one submission per group. Your submission will be graded as a group.
>  1. Include your group number and the names of all group members in the signature block at the top of this notebook.
>  2. In the cell below, paste the link to your project repository. One link per group. The grader will review the activity and history provided by GitHub. To add a hyperlink to a Jupyter markdown cell, follow the instructions in the cell below.
>  3. Below the GitHub Repository Link cell is a code cell. Copy and paste your final program code into this cell.

#### GitHub Repository Link
> [Github Repo](https://github.com/chrissor-umd/INST326_Group_54)

In [ ]:
# Solution - enter your code solution below
import tkinter as tk
import datetime as dt # For handling data and time calculations
from datetime import time
import json # For saving and loading data

# Stuff used for the calendar
import calendar
from IPython.display import HTML

# Stuff used for pay reports
import csv


class ScheduleManager():
    def __init__(self, day_start=time(0,0), day_end=time(23,59), shift_length_minutes=30):
        self.day_start = day_start # Earliest possible shift
        self.day_end = day_end # No more shifts will be generated past this time
        self.shift_length = shift_length_minutes # How long a shift is in minutes. When new shifts are added, the given time interval is broken down into shifts of this many minutes, which are each added to the schedule
        self.day_shift_times = self.__generate_day_shift_times()
        self.__load_json() # Loads necessary data for managing people and schedules


    #############################################################################################
    #######    SUPER AWESOME AND NOT ANNOYING JSON ENCODING STUFF I SPENT 6 HOURS ON :)    ######
    #############################################################################################

    @staticmethod
    def __custom_object_hook(obj): # Converts JSON objects into the appropriate custom classes
        if "is_person" in obj:
            name = obj["name"]
            phone = obj["phone"]
            email = obj["email"]
            person = Person(name, phone, email)
            for attr in obj.keys():
                if attr not in {'name','phone','email'}:
                    setattr(person, attr, obj[attr])
            return person
        else:
            if isinstance(obj, str):
                ScheduleManager.convert_from_iso(obj)
            return obj
    
    @staticmethod
    def __custom_encode(obj):
        if isinstance(obj, Person):
            return obj.__dict__ # Converts a person into a serializable dictionary
        elif isinstance(obj, time) or isinstance(obj, dt.datetime) or isinstance(obj, dt.date):
            return obj.isoformat()
        elif isinstance(obj, dt.datetime):
            return
        else:
            return super().default(obj)
        
    @staticmethod
    def convert_to_strings(dictionary):
        """Converts all keys in the data dictionaries to strings for encoding"""
        return_dict = {}
        for _, (key, value) in enumerate(dictionary.items()):
            if isinstance(key, time) or isinstance(key, dt.datetime) or isinstance(key, dt.date):
                key = key.isoformat()
            if isinstance(value, dict):
                value = ScheduleManager.convert_to_strings(value)
            elif isinstance(value, Person):
                value = value.__dict__
                value = ScheduleManager.convert_to_strings(value)
            return_dict.update({key:value}) # Recursive
        return return_dict
    
    @staticmethod
    def convert_from_strings(dictionary):
        """Converts dictionary keys back into time format"""
        return_dict = {}
        for _, (key, value) in enumerate(dictionary.items()):
            if isinstance(key, str):
                key = ScheduleManager.convert_from_iso(key)
            if isinstance(value, dict):
                value = ScheduleManager.convert_from_strings(value)
            elif isinstance(value, Person):
                for i, s in enumerate(value.shift_times):
                    value.shift_times[i] = ScheduleManager.convert_from_iso(s)
                setattr(value, '_Person__shift_exceptions', ScheduleManager.convert_from_strings(getattr(value, '_Person__shift_exceptions')))
                setattr(value, '_Person__shift_availability', ScheduleManager.convert_from_strings(getattr(value, '_Person__shift_availability')))
            return_dict.update({key:value})
        return return_dict
                        
    @staticmethod
    def convert_from_iso(iso):
        """Tries to convert iso formatted datetime object into its original class"""
        if isinstance(iso, str):
            try:
                datetime_obj = dt.datetime.fromisoformat(iso)
                datetime_obj.replace(second=0, microsecond=0)
                return datetime_obj
            except (TypeError, ValueError):
                try:
                    date_obj = dt.date.fromisoformat(iso)
                    return date_obj
                except (TypeError, ValueError):
                    try:
                        time_obj = time.fromisoformat(iso)
                        time_obj.replace(second=0, microsecond=0)
                        return time_obj
                    except (TypeError, ValueError):
                        return iso
                
    def __load_json(self):
        """Loads stored data from json files"""
        try:
            with open('people_data.json', 'r') as people_file:
                people_data = people_file.read()
                if len(people_data) > 0:
                    people_dict = json.loads(people_data, object_hook=self.__custom_object_hook) # For some reason, I can only get this to work right by storing and loading it as a string like this
                    self.__people = ScheduleManager.convert_from_strings(people_dict)
                else:
                    self.__people = {}
                print("Loaded people_data.json")
        except FileNotFoundError:
            with open('people_data.json', 'w') as people_file:
                self.__people = {}
                json.dump(self.__people, people_file, default=self.__custom_encode)
                print("Created people_data.json")
        try:
            with open('shifts_data.json', 'r') as shifts_file:
                shifts = shifts_file.read()
                if len(shifts) > 0:
                    shifts_dict = json.loads(shifts, object_hook=self.__custom_object_hook)
                    self.__shifts = ScheduleManager.convert_from_strings(shifts_dict)
                    print("Loaded shifts_data.json")
                else:
                    self.__shifts = self.__generate_empty_shifts()
        except FileNotFoundError:
            with open('shifts_data.json', 'w') as shifts_file:
                self.__shifts = self.__generate_empty_shifts()
                shifts_dict = ScheduleManager.convert_to_strings(self.__shifts)
                json.dump(shifts_dict, shifts_file, default=self.__custom_encode)
                print("Created shifts_data.json")

    def __save_json(self):
        """Handles data conversions for smooth json serialization, and loads data to file"""
        people_dict = ScheduleManager.convert_to_strings(self.__people)
        shifts_dict = ScheduleManager.convert_to_strings(self.__shifts)
        with open('people_data.json', 'w') as people_file:
            json_dump = json.dumps(people_dict, indent=4, default=self.__custom_encode)
            people_file.write(json_dump)
        with open('shifts_data.json', 'w') as shifts_file:
            json_dump = json.dumps(shifts_dict, indent=4, default=self.__custom_encode)
            shifts_file.write(json_dump)
            
    ################################################
    #######         END OF JSON STUFF         ######
    ################################################


    def add_person(self, name, phone=None, email=None):
        """Creates a new person object with a unique id"""
        person_id = name.upper()
        person = Person(name, phone, email) # Creates the person object
        if person_id in self.__people.keys():
            print("Person already exists")
            return person
        person.shift_length_minutes = self.shift_length
        person.shift_times = self.day_shift_times.copy() # Passes shift times to Person for handling availability
        person.create_availability_dict()
        self.__people.update({person_id:person}) # Stores the new person in the people dictionary, with their name as the key
        print(f"Added {person.name}")
        self.__save_json()
        return person
    
    def get_person(self, name): 
        """Get the person object associated with a given name or person_id"""
        if name.upper() in self.__people:
            return self.__people[name.upper()]
        else:
            print("Person not found")

    def get_person_id(self, person_or_name):
        """Gets a formatted person_id from either their name or a person object"""
        if isinstance(person_or_name, Person):
            person_id = person_or_name.name.upper()
        elif isinstance(person_or_name, str):
            person_id = person_or_name.upper()
            if person_id not in self.__people.keys():
                raise Exception("Person does not exist")
        return person_id

    def remove_person(self, person_id):
        """Removes person from ScheduleManager"""
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        for day in self.__shifts.keys():
            self.unassign_shift(person_id, day, time(0), time(23,59)) # Removes the person from any shifts they're assigned to
        self.__people.pop(person_id) # Removes person from list
        print(f"{person.name} removed")
        self.__save_json()
        return

    def assign_shift(self, person_id, day, start, end=None):
        """Assigns given person_id to all shifts in range"""
        day, time_start, time_end = self.get_formatted_args(day, start, end)
        if time_end == None:
            time_end = self.add_time(start, self.shift_length)
        if time_start < self.day_start or time_end > self.day_end:
            print("Time outside of available hours")
            return
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        shift_added = False # Both of these are just flags for more accurate print statements
        shift_found = False
        time_start = self.__get_shift_from_time(time_start)
        for shift_start in self.__shifts[day].keys(): # Get list of shifts
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within the given time interval
                shift_found = True
                if person_id not in self.__shifts[day][shift_start]: # Prevent duplicate entries
                    self.__shifts[day][shift_start].append(person_id)
                    shift_added = True
        if shift_added:
            self.__save_json()
            print(f"Added {person.name} to shifts")
        else:
            if shift_found:
                print("Person already assigned to shifts")  # In case they're already fully booked for that time frame
            else:
                print("Could not assign shift")

    def __get_shift_from_time(self, time_to_check):
        """Gets the shift that contains a given time"""
        time_val = ScheduleManager.get_formatted_args(time_to_check)
        if time_val < self.day_start or time_val > self.day_end:
            return time_to_check
        for shift_start in self.__generate_day_shift_times():
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_val, time_val)):
                return shift_start
            
    def unassign_shift(self, person_id, day, start, end=None):
        """Removes given person_id from all shifts in range"""
        day, time_start, time_end = self.get_formatted_args(day, start, end)
        if time_end is None:
            time_end = ScheduleManager.add_time(time_start, self.shift_length) # If you only want to modify a single shift
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        shift_removed = False
        for shift_start in self.__shifts[day].keys(): # Get list of shifts
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within the given time interval
                if person_id in self.__shifts[day][shift_start]: # Prevent duplicate entries
                    self.__shifts[day][shift_start].remove(person_id)
                    shift_removed = True
        if shift_removed:
            self.__save_json()
            print(f"Removed {person.name} from shifts")

    def set_availability(self, person_id, availability, day, start, end=None):
        """Set availability for a specific person"""
        day, start, end = ScheduleManager.get_formatted_args(day, start, end)
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        if end is None:
            end = ScheduleManager.add_time(start, 30)  # If you only want to modify a single shift
        person.set_shift_availability(availability, day, start, end)
        self.__save_json()

    def add_availability_exception(self, person_id, availability, date, start, end=None):
        """Add availability exception for a specific person"""
        date, start, end = ScheduleManager.get_formatted_args(date, start, end)
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        if end is None:
            end = self.__get_default_end_time(start)  # If you only want to modify a single shift
        start = self.__get_shift_from_time(start)
        person.add_shift_exception(availability, date, start, end)

    def get_availability(self, person_id, *args):
        """Gets availability status of a specific person on a given time and date"""
        if len(args) == 2:
            date, shift = ScheduleManager.get_formatted_args(args[0], args[1])
        elif len(args) == 1:
            datetime = ScheduleManager.get_formatted_args(args[0])
            date = datetime.date()
            shift = datetime.time()
        else:
            raise Exception("Arguments must be both 'date' and 'time', or 'datetime'")
        person_id = self.get_person_id(person_id)
        person = self.get_person(person_id)
        shift = self.__get_shift_from_time(shift)
        return person.get_availability(date, shift)
    
    def __generate_day_shift_times(self):
        """Calculates and returns a list of shift times for a day"""
        start_shift = self.day_start
        current_shift = start_shift
        shift_time_list = []
        while current_shift < self.day_end:
            shift_time_list.append(current_shift)
            current_shift = ScheduleManager.add_time(current_shift, self.shift_length)
        return shift_time_list

    def __generate_empty_shifts(self):
        """Returns dictionary with empty shifts for each day"""
        shift_time_list = self.day_shift_times
        days = {'SUN','MON','TUE','WED','THU','FRI','SAT'} # Some of this is probably redundant but I don't want any weird indexing issues
        shifts_dict = {'SUN':{},'MON':{},'TUE':{},'WED':{},'THU':{},'FRI':{},'SAT':{}} # Each day is a dictionary that will contain a day and an associated dictionary of shifts, each containing a list of scheduled people
        for day in days:
            for s in shift_time_list:
                shifts_dict[day].update({s:[]})
        print("Generated empty shifts")
        return shifts_dict

    @staticmethod
    def add_time(base_time, minutes_to_add):
        """Datetime doesn't have a way to add time to time objects, so this handles that. Results that exceed 24 hours are capped at 23:59"""
        base_time = ScheduleManager.get_formatted_args(base_time)
        null_date = dt.datetime(2000, 1, 1) # Null date just so the addition works
        base_time = dt.datetime.combine(null_date, base_time)
        delta = dt.timedelta(minutes=minutes_to_add)
        new_datetime = base_time + delta
        new_time = new_datetime.time()
        if new_datetime.day == 1:
            new_time.replace(second=0, microsecond=0)
            return new_time
        else:
            return time(23,59) # Default to 23:59 if the hours overflow into the next day

    @staticmethod
    def check_overlap(interval1, interval2):
        """Checks if there is any kind of overlap between two time intervals, and returns the type of overlap as a str, or False if no overlap is present"""
        time_list = [interval1[0],interval1[1],interval2[0],interval2[1]]
        int1_start = time_list[0]
        int1_end = time_list[1]
        int2_start = time_list[2]
        int2_end = time_list[3]
        if int1_start > int1_end or int2_start > int2_end: # start time should always be less than end time
            raise Exception("Invalid time input")
        if int1_end < int2_start or int1_start > int2_end: # No overlap
            return False
        elif int1_start >= int2_start and int1_end <= int2_end: # Full overlap
            return 'inside'
        elif int1_start <= int2_start and int1_end >= int2_end:
            return 'outside'
        elif int1_start > int2_start: # Left overlap
            return 'right'
        elif int1_end < int2_end: # Right overlap
            return 'left'

    @staticmethod
    def get_formatted_args(*args):
        """Returns a formatted and typechecked tuple of arguments, raising error on invalid input. Day can be single day as string, or list of strings"""
        return_list = []
        for arg in args:
            if isinstance(arg, time):
                time_ = arg
                time_.replace(second=0, microsecond=0)
                return_list.append(time_)
            elif isinstance(arg, str):
                day = arg
                day = day[0:3].upper() # Gets the uppercase forms of the first 3 letters of day, so inputs like 'Sunday' and 'sun' are both accepted
                if day not in {'SUN','MON','TUE','WED','THU','FRI','SAT'}:
                    raise Exception(f"Invalid day '{day}'")
                else:
                    return_list.append(day)
            elif isinstance(arg, list):
                days = arg
                for d in enumerate(days):
                    index = d[0]
                    current_day = d[1]
                    if type(current_day) != str or current_day not in {'SUN','MON','TUE','WED','THU','FRI','SAT'}:
                        raise Exception("Invalid day format")
                    days[index] = current_day[0:3].upper() # Gets the uppercase forms of the first 3 letters of day, so inputs like 'Sunday' and 'sun' are both accepted
                return_list.append(days)
            elif isinstance(arg, dt.date):
                return_list.append(arg)
            elif isinstance(arg, dt.datetime):
                datetime = arg
                datetime.replace(minute=0, second=0)
                return_list.append(datetime)
            elif arg is None:
                return_list.append(arg) # So this doesn't break on optional values
        if len(return_list) < 2: # Don't return a tuple of 1 value
            if len(return_list) == 0:
                return None
            return return_list[0]
        return tuple(return_list) # Returns a tuple containing the formatted and typechecked inputs


class Person:
    def __init__(self, name, phone=None, email=None):
        self.is_person = True # This helps with JSON decoding
        self.name = str(name)
        self.phone = phone # Optional
        self.email = email # Optional
        self.__pay_rate = 0
        self.__shift_exceptions = {}
        self.shift_length_minutes = 30 # Defaults to 30 minute time intervals, should be passed specific value by ScheduleManager
        self.shift_times = [] # Default timing of shifts passed from the ScheduleManager

    def __str__(self):
        """Returns name of person when object is converted to string"""
        return str(self.name)
    
    @property
    def pay_rate(self):
        """Read Only"""
        return self.__pay_rate

    def set_pay_rate(self, new_rate):
        """Sets the new rate, checking the type and rejecting negative numbers"""
        if type(new_rate) in {float, int}:
            if new_rate >= 0:
                self.__pay_rate = new_rate
            else:
                print("New rate must be positive number or zero")
        else:
            print("Invalid pay rate amount")

    def create_availability_dict(self):
        days = {'SUN','MON','TUE','WED','THU','FRI','SAT'} # Some of this is probably redundant but I don't want any weird indexing issues
        self.__shift_availability = {'SUN':{},'MON':{},'TUE':{},'WED':{},'THU':{},'FRI':{},'SAT':{}} # Each day is a dictionary that will contain a day and an associated dictionary of shifts, each containing a list of scheduled people
        for day in days:
            for s in self.shift_times:
                self.__shift_availability[day].update({s:'available'})
        print(f"Created availability dictionary for {self.name}")

    def set_shift_availability(self, availability, day, start, end):
        """Changes the availability for the given day and time"""
        availability = availability.lower()
        if availability not in {'preferred', 'available', 'unavailable'}: # Check for valid availability
            print(f"Invalid availability '{availability}', please select only from: 'available', 'unavailable', 'preferred'")
            return
        day, time_start, time_end = ScheduleManager.get_formatted_args(day, start, end)
        availability_dict = self.__shift_availability[day].copy()
        for shift_start in availability_dict.keys(): # Get list of shifts
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length_minutes)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within the given time interval
                self.__shift_availability[day].update({shift_start:availability})
        print(f"Changed availability to '{availability}' for shifts from {time_start.isoformat('minutes')} to {time_end.isoformat('minutes')}")

    def add_shift_exception(self, availability, date, start, end, recurring=False):
        """Adds an availaibility exeption for a specific date and range of time. Set recurring to True for it to apply for every year"""
        if availability not in {'preferred', 'available', 'unavailable'}: # Check for valid availability
            print(f"Invalid availability '{availability}', please select only from: 'available', 'unavailable', 'preferred'")
            return
        time_start, time_end = ScheduleManager.get_formatted_args(start, end)
        if isinstance(date, dt.date):
            pass
        elif isinstance(date, dt.datetime):
            date = date.date()
        else:
            print("Invalid date")
            return
        if date not in self.__shift_exceptions:
            self.__shift_exceptions.update({date:{}})
        date_shift_exceptions = self.__shift_exceptions[date]
        preference_changed = False
        for shift_start in self.shift_times:
            shift_end = ScheduleManager.add_time(shift_start, self.shift_length_minutes)
            if ScheduleManager.check_overlap((shift_start, shift_end), (time_start, time_end)): # Check if shift is within given time range
                date_shift_exceptions.update({shift_start:[availability, recurring]})
                preference_changed = True
        if preference_changed:
            recurring_msg = ""
            if recurring:
                recurring_msg = "recurring"
            print(f"Added {recurring_msg} shift exception to '{availability}' from {time_start} to {time_end}")
        else:
            if not self.__shift_exceptions[date]:
                self.__shift_exceptions.pop(date)

    def update_exceptions(self):
        """Updates all recurring shift exceptions to apply to the current year, and cleans old shift exceptions"""
        current_year = dt.datetime.now().year
        for date in self.__shift_exceptions.keys():
            date_exceptions = self.__shift_exceptions[date]
            for shift in date_exceptions.keys():
                status_list = date_exceptions[shift]
                recurring = status_list[1] # Check if exception is recurring
                if date.year < current_year:
                    if recurring:
                        self.__shift_exceptions.pop(date) # Remove old entry
                        date.replace(year=current_year) # Updates year
                        self.__shift_exceptions.update({date:[status_list]}) # Replaces old entry with new one
                    else:
                        self.__shift_exceptions.pop(date) # Removes old entry
        
    def get_availability(self, date, shift):
        self.update_exceptions()
        if isinstance(date, dt.date):
            pass
        elif isinstance(date, dt.datetime):
            date = date.date()
        else:
            print("Invalid date")
            return
        work_end = ScheduleManager.add_time(self.shift_times[-1], self.shift_length_minutes) # Calculate when the last shift would end
        if shift < self.shift_times[0] or shift > work_end:
            return 'unavailable'
        if shift not in self.shift_times: # Check if the shift availability needs to be calculated
            for i, t in enumerate(self.shift_times): # If the given time is not a shift starting time, calculate availability based on which shift the given time is in
                if i < len(self.shift_times)-1: # Check to avoid exceeding the list length
                    current_shift = self.shift_times[i]
                    next_shift = self.shift_times[i+1]
                    if current_shift <= shift < next_shift: # Check between which two shifts the given time falls under
                            shift = current_shift
                            break
                else:
                    last_shift = self.shift_times[-1]
                    if last_shift <= shift < work_end:
                        shift = last_shift
                    else:
                        return 'unavailable'
        if date in self.__shift_exceptions.keys() and shift in self.__shift_exceptions[date].keys(): # Check if the date has an availability exception
            availability_dict = self.__shift_exceptions[date]
            return availability_dict[shift][0]
        else:
            days = ['MON','TUE','WED','THU','FRI','SAT','SUN'] # date.weekday() has 0 set as Monday, for some reason
            day = days[date.weekday()] # Gets the day of the week from the given date
            availability_dict = self.__shift_availability[day]
            return availability_dict[shift]

class WorkSchedule:
    def __init__(self):
        self.employees = {}
        self.shifts = {}

    def add_employee(self, name: str) -> None:
        """Add a new employee to the schedule."""
        name = name.strip().title()
        if name not in self.employees:
            self.employees[name] = name.upper()
            print(f"Employee '{name}' added.")
        else:
            print(f"Employee '{name}' already exists.")

    def assign_shift(self, name: str, day: str, start_time: str, end_time: str) -> None:
        """Assign a shift to an employee on a specific day."""
        name = name.strip().title()
        if name not in self.employees:
            print(f"Error: Employee '{name}' not found.")
            return
        
        day = day.strip().lower()
        if day not in self.shifts:
            self.shifts[day] = {}
        if name.upper() not in self.shifts[day]:
            self.shifts[day][name.upper()] = []
        
        self.shifts[day][name.upper()].append((start_time, end_time))
        print(f"Shift added for '{name}' on {day.capitalize()} from {start_time} to {end_time}.")

    def calculate_hours(self, start_time: str, end_time: str) -> float:
        """Calculate hours worked between start and end times."""
        from datetime import datetime
        time_format = "%H:%M"
        start = datetime.strptime(start_time, time_format)
        end = datetime.strptime(end_time, time_format)
        delta = (end - start).seconds / 3600.0
        return delta

    def generate_pay_report(self, hourly_rate: float = 20.0) -> None:
        """Generate and save the pay report with only pay details, excluding shifts or employee data."""
        report_filename = "pay_report.csv"
        
        with open(report_filename, mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(["Employee Name", "Total Hours Worked", "Weekly Pay", "Monthly Pay"])

            for name in self.employees.values():
                # Calculate total hours worked by summing hours for each shift assigned to the employee
                total_hours = 0
                for day, shifts in self.shifts.items():
                    if name.upper() in shifts:
                        for shift in shifts[name.upper()]:
                            start_time, end_time = shift
                            total_hours += self.calculate_hours(start_time, end_time)
                
                # Calculate weekly and monthly pay
                weekly_pay = total_hours * hourly_rate
                monthly_pay = weekly_pay * 4  # Assuming 4 weeks per month for simplicity
                
                # Write the pay details to the CSV report
                writer.writerow([name, total_hours, weekly_pay, monthly_pay])
        
        print(f"Pay report saved to {report_filename}.")

if __name__ == "__main__":
# Example Usage
    work_schedule = WorkSchedule()
    work_schedule.add_employee("Alice")
    work_schedule.add_employee("Bob")

    # Assign shifts
    work_schedule.assign_shift("Alice", "Monday", "09:00", "17:00")
    work_schedule.assign_shift("Alice", "Wednesday", "09:00", "17:00")
    work_schedule.assign_shift("Bob", "Tuesday", "10:00", "18:00")

    # Generate pay report (just pay-related data, no shift data)
    work_schedule.generate_pay_report(hourly_rate=25.0)


#Calendar stuff
class CalendarApp:
    def __init__(self, root):
        self.calendar_manager = CalendarManager()
        self.root = root
        self.root.title("Calendar Viewer")
        self.setup_ui()

    def setup_ui(self):
        
        self.text_widget = tk.Text(self.root, wrap="none", width=100, height=30)
        self.text_widget.grid(row=0, column=0, columnspan=2)

        
        self.year_button = tk.Button(self.root, text="View Year", command=self.display_year)
        self.year_button.grid(row=1, column=0, padx=5, pady=5)

        self.month_button = tk.Button(self.root, text="View Month", command=self.display_month)
        self.month_button.grid(row=1, column=1, padx=5, pady=5)

        
        self.month_entry = tk.Entry(self.root, width=5)
        self.month_entry.grid(row=2, column=1, padx=5, pady=5)
        tk.Label(self.root, text="Enter Month (1-12):").grid(row=2, column=0, padx=5, pady=5)

    def display_year(self):
       
        self.calendar_manager.set_year_view()
        calendar_text = self.calendar_manager.render_calendar()
        self.update_text_widget(calendar_text)

    def display_month(self):
        
        month = self.month_entry.get()
        if month.isdigit() and 1 <= int(month) <= 12:
            self.calendar_manager.set_month_view(int(month))
            calendar_text = self.calendar_manager.render_calendar()
            self.update_text_widget(calendar_text)
        else:
            self.update_text_widget("Invalid month! Please enter a number between 1 and 12.")

    def update_text_widget(self, content):
        
        self.text_widget.delete("1.0", tk.END)  # Clear existing content
        self.text_widget.insert(tk.END, content)

class CalendarManager:
    def __init__(self):
        self.year = 2024  
        self.month = None  
        self.text_cal = calendar.TextCalendar()

    def set_year_view(self):
        
        self.month = None

    def set_month_view(self, month):
        """Switch to month view."""
        if 1 <= month <= 12:
            self.month = month
        else:
            raise ValueError("Month must be between 1 and 12.")

    def render_calendar(self):
        
        if self.month: 
            return self.text_cal.formatmonth(self.year, self.month)
        else:  
            return self.text_cal.formatyear(self.year)

# Run the Tkinter app
if __name__ == "__main__":
    root = tk.Tk()
    app = CalendarApp(root)
    root.mainloop()

 


calendar_manager = CalendarManager()




if __name__ == "__main__":
    schedule = ScheduleManager(time(7), time(19), 360)
    person1 = schedule.add_person('Joe')
    person2 = schedule.add_person('Bob')
    schedule.remove_person('Bob')
    schedule.assign_shift(person1, 'sun', time(10,30),time(17,00))
    schedule.unassign_shift(person1, 'sun', time(10,30))
    schedule.add_availability_exception(person1, 'preferred', dt.date(2024, 11, 20), time(14),time(23))
    schedule.set_availability(person1, 'preferred', 'sun', time(5,30))
    now = dt.datetime.now()
    now_date = now.date()
    now_time = now.time()
    print(schedule.get_availability(person1, now_date, time(15)))


Employee 'Alice' added.
Employee 'Bob' added.
Shift added for 'Alice' on Monday from 09:00 to 17:00.
Shift added for 'Alice' on Wednesday from 09:00 to 17:00.
Shift added for 'Bob' on Tuesday from 10:00 to 18:00.
Pay report saved to pay_report.csv.
Loaded people_data.json
Loaded shifts_data.json
Person already exists
Created availability dictionary for Bob
Added Bob
Bob removed
Added Joe to shifts
Removed Joe from shifts
Added  shift exception to 'preferred' from 13:00:00 to 23:00:00
Changed availability to 'preferred' for shifts from 05:30 to 06:00
preferred


### Notebook Instructions
> Before turning in your notebook:
> 1. Make sure you have renamed the notebook file as instructed
> 2. Make sure you have included your signature block and that it is correct according to the instructions
> 3. comment your code as necessary
> 4. run all code cells and double check that they run correctly. If you can't get your code to run correctly and you want partial credit, add a note for the grader in a new markdown cell directly above your code solution.<br><br>
Turn in your notebook by uploading it to ELMS<br>
IF the exercises involve saved data files, put your notebook and the data file(s) in a zip folder and upload the zip folder to ELMS